In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave

Using TensorFlow backend.
/home/hongta/anaconda3/envs/rcnn/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
import numpy as np
import random
import tensorflow as tf

In [4]:
DATASET_PATH = './dataset'

In [5]:
# Get images
X = []
for filename in os.listdir(f"{DATASET_PATH}/train"):
    X.append(img_to_array(load_img(f"{DATASET_PATH}/train/{filename}")))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.95 * len(X))
X_train = X[:split]
X_train = 1.0/255 * X_train

In [6]:
# Build a model
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

In [7]:
model.compile(optimizer='rmsprop',
              loss='mse')

In [8]:
# Image transformer
datagen = ImageDataGenerator(shear_range=0.2,
                             zoom_range=0.2,
                             rotation_range=20,
                             horizontal_flip=True)

# Generate training data
def image_a_b_gen(batch_size):
    for batch in datagen.flow(X_train, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:, :, :, 0]
        y_batch = lab_batch[:, :, :, 1:] / 128
        
        yield (X_batch.reshape(X_batch.shape + (1, )), y_batch)

In [21]:
batch_size = 10
tensorboard = TensorBoard(log_dir="./log/beta")
model.fit_generator(image_a_b_gen(batch_size), 
                    callbacks=[tensorboard], 
                    epochs=500, 
                    steps_per_epoch=10,
                    verbose=True)

Epoch 1/500
10/10 [==============================] - 5s 501ms/step - loss: 0.0011
Epoch 2/500
10/10 [==============================] - 5s 516ms/step - loss: 0.0021
Epoch 3/500
10/10 [==============================] - 5s 516ms/step - loss: 9.9078e-04
Epoch 4/500
10/10 [==============================] - 5s 515ms/step - loss: 0.0010
Epoch 5/500
10/10 [==============================] - 5s 508ms/step - loss: 0.0014
Epoch 6/500
10/10 [==============================] - 5s 513ms/step - loss: 9.6199e-04
Epoch 7/500
10/10 [==============================] - 5s 516ms/step - loss: 0.0014
Epoch 8/500
10/10 [==============================] - 5s 514ms/step - loss: 0.0013
Epoch 9/500
10/10 [==============================] - 5s 523ms/step - loss: 9.3804e-04
Epoch 10/500
10/10 [==============================] - 5s 513ms/step - loss: 0.0010
Epoch 11/500
10/10 [==============================] - 5s 512ms/step - loss: 0.0012
Epoch 12/500
10/10 [==============================] - 5s 521ms/step - loss: 9.9268e-

10/10 [==============================] - 5s 509ms/step - loss: 6.2036e-04
Epoch 97/500
10/10 [==============================] - 5s 513ms/step - loss: 7.6171e-04
Epoch 98/500
10/10 [==============================] - 5s 513ms/step - loss: 7.3858e-04
Epoch 99/500
10/10 [==============================] - 5s 505ms/step - loss: 6.3854e-04
Epoch 100/500
10/10 [==============================] - 5s 511ms/step - loss: 0.0020
Epoch 101/500
10/10 [==============================] - 5s 507ms/step - loss: 8.8541e-04
Epoch 102/500
10/10 [==============================] - 5s 510ms/step - loss: 6.3125e-04
Epoch 103/500
10/10 [==============================] - 5s 511ms/step - loss: 6.2744e-04
Epoch 104/500
10/10 [==============================] - 5s 511ms/step - loss: 6.1214e-04
Epoch 105/500
10/10 [==============================] - 5s 512ms/step - loss: 6.3291e-04
Epoch 106/500
10/10 [==============================] - 5s 516ms/step - loss: 0.0013
Epoch 107/500
10/10 [==============================] - 5s

10/10 [==============================] - 5s 508ms/step - loss: 7.0097e-04
Epoch 191/500
10/10 [==============================] - 5s 500ms/step - loss: 0.0010
Epoch 192/500
10/10 [==============================] - 5s 498ms/step - loss: 5.6315e-04
Epoch 193/500
10/10 [==============================] - 5s 502ms/step - loss: 5.5116e-04
Epoch 194/500
10/10 [==============================] - 5s 506ms/step - loss: 5.9323e-04
Epoch 195/500
10/10 [==============================] - 5s 504ms/step - loss: 7.6447e-04
Epoch 196/500
10/10 [==============================] - 5s 504ms/step - loss: 0.0012
Epoch 197/500
10/10 [==============================] - 5s 508ms/step - loss: 5.4261e-04
Epoch 198/500
10/10 [==============================] - 5s 510ms/step - loss: 6.2788e-04
Epoch 199/500
10/10 [==============================] - 5s 501ms/step - loss: 5.9290e-04
Epoch 200/500
10/10 [==============================] - 5s 501ms/step - loss: 5.8604e-04
Epoch 201/500
10/10 [==============================] -

10/10 [==============================] - 5s 495ms/step - loss: 5.6328e-04
Epoch 284/500
10/10 [==============================] - 5s 498ms/step - loss: 5.1211e-04
Epoch 285/500
10/10 [==============================] - 5s 500ms/step - loss: 0.0017
Epoch 286/500
10/10 [==============================] - 5s 494ms/step - loss: 5.9861e-04
Epoch 287/500
10/10 [==============================] - 5s 494ms/step - loss: 6.3214e-04
Epoch 288/500
10/10 [==============================] - 5s 493ms/step - loss: 5.6840e-04
Epoch 289/500
10/10 [==============================] - 5s 494ms/step - loss: 5.5123e-04
Epoch 290/500
10/10 [==============================] - 5s 496ms/step - loss: 5.4774e-04
Epoch 291/500
10/10 [==============================] - 5s 494ms/step - loss: 5.2926e-04
Epoch 292/500
10/10 [==============================] - 5s 495ms/step - loss: 5.2772e-04
Epoch 293/500
10/10 [==============================] - 5s 497ms/step - loss: 0.0010
Epoch 294/500
10/10 [==============================] -

Epoch 377/500
10/10 [==============================] - 5s 503ms/step - loss: 5.6185e-04
Epoch 378/500
10/10 [==============================] - 5s 497ms/step - loss: 0.0011
Epoch 379/500
10/10 [==============================] - 5s 500ms/step - loss: 5.3823e-04
Epoch 380/500
10/10 [==============================] - 5s 497ms/step - loss: 5.2340e-04
Epoch 381/500
10/10 [==============================] - 5s 499ms/step - loss: 5.1760e-04
Epoch 382/500
10/10 [==============================] - 5s 501ms/step - loss: 6.6745e-04
Epoch 383/500
10/10 [==============================] - 5s 501ms/step - loss: 7.5337e-04
Epoch 384/500
10/10 [==============================] - 5s 500ms/step - loss: 5.2650e-04
Epoch 385/500
10/10 [==============================] - 5s 502ms/step - loss: 5.3311e-04
Epoch 386/500
10/10 [==============================] - 5s 500ms/step - loss: 5.1830e-04
Epoch 387/500
10/10 [==============================] - 5s 500ms/step - loss: 0.0022
Epoch 388/500
10/10 [===================

10/10 [==============================] - 5s 497ms/step - loss: 4.8033e-04
Epoch 471/500
10/10 [==============================] - 5s 490ms/step - loss: 6.7014e-04
Epoch 472/500
10/10 [==============================] - 5s 496ms/step - loss: 4.7705e-04
Epoch 473/500
10/10 [==============================] - 5s 500ms/step - loss: 4.9979e-04
Epoch 474/500
10/10 [==============================] - 5s 501ms/step - loss: 5.2113e-04
Epoch 475/500
10/10 [==============================] - 5s 499ms/step - loss: 6.5251e-04
Epoch 476/500
10/10 [==============================] - 5s 497ms/step - loss: 4.9142e-04
Epoch 477/500
10/10 [==============================] - 5s 500ms/step - loss: 4.8781e-04
Epoch 478/500
10/10 [==============================] - 5s 498ms/step - loss: 4.9906e-04
Epoch 479/500
10/10 [==============================] - 5s 497ms/step - loss: 8.2956e-04
Epoch 480/500
10/10 [==============================] - 5s 499ms/step - loss: 4.7936e-04
Epoch 481/500
10/10 [=========================

In [22]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model/beta.h5")

In [23]:
# Test model
X_test = rgb2lab(1.0 / 255 * X[split:])[:, :, :, 0]
X_test = X_test.reshape(X_test.shape + (1, )) # from (1, 256, 256) => (1, 256, 256, 1)
y_test = rgb2lab(1.0 / 255 * X[split:])[:, :, :, 1:]
y_test = y_test / 128
print(model.evaluate(X_test, y_test, batch_size=batch_size))

1/1 [==============================] - 0s 26ms/step
0.00794215686619


In [24]:
color_me = []
for filename in os.listdir(f"{DATASET_PATH}/test/"):
    color_me.append(img_to_array(load_img(f"{DATASET_PATH}/test/{filename}")))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0 / 255 * color_me)[:, :, :, 0]
color_me = color_me.reshape(color_me.shape + (1, ))

In [25]:
# Test model
output = model.predict(color_me)
output = output * 128

for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:, :, 0] = color_me[i][:, :, 0]
    cur[:, :, 1:] = output[i]
    imsave(f"result/img_{str(i)}.png", lab2rgb(cur))

/home/hongta/anaconda3/envs/rcnn/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
